# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", 'w')

In [3]:
output.write('123')

3

In [4]:
output.close()

In [6]:
sys.stdout.write('1')
sys.stdout.write('2')

12

1

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [14]:
from datetime import datetime
import sys 

original_write = sys.stdout.write

def my_write(string_text):
    if not string_text.strip():
        return 0
    new_string = f'[{datetime.now().strftime("%Y-%m-%d %H:%M:%S")} ]: {string_text}'
    n_bytes = original_write(new_string)
    return n_bytes

sys.stdout.write = my_write

In [15]:
sys.stdout.write('1, 2, 3')

[2022-11-23 09:04:10 ]: 1, 2, 3

31

In [16]:
print('1, 2, 3')
sys.stdout.write = original_write

[2022-11-23 09:04:11 ]: 1, 2, 3

In [17]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

### Первый способ

In [14]:
def timed_output(transform):
    def decorator(function):
        def wrapper(name):
            original_write = sys.stdout.write
            sys.stdout.write = transform
            function(name)
            sys.stdout.write = original_write    
        return wrapper
    return decorator

In [15]:
@timed_output(my_write)
def print_greeting(name):
    print(f'Hello, {name}!')

In [16]:
print_greeting("Nikita")

[2022-11-22 13:01:18]: Hello, Nikita![2022-11-22 13:01:18]: 


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

### Второй способ

Я не понимаю, почему возникает различие с первым способом. Почему дважды печатается метка времени?

In [21]:
def timed_output(function):
    def wrapper(name):
        original_write = sys.stdout.write
        
        def my_write(string_text):
            #if not string_text.strip():
            #    return 0
            new_string = f'[{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}]: ' + string_text
            n_bytes = original_write(new_string)
            return n_bytes

        sys.stdout.write = my_write
        function(name)
        sys.stdout.write = original_write    
    return wrapper

In [22]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [23]:
print_greeting("Nikita")

[2022-11-23 09:07:24]: Hello, Nikita![2022-11-23 09:07:24]: 


## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [103]:
def redirect_output(filepath):
    def decorator(func):
        def wrapper():
            original_out = sys.stdout
            sys.stdout = open(filepath, 'a')
            result = func()
            sys.stdout.close()
            sys.stdout = original_out
            return result
        return wrapper
    return decorator
            
            

In [29]:
from functools import wraps


def redirect_output(filepath):
    def decorator(func):
        @wraps(func)
        def wrapper():
            original_out = sys.stdout
            with open(filepath, 'a') as f:
                sys.stdout = f
                result = func()
            sys.stdout = original_out
            return result
        return wrapper
    return decorator

In [30]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power)
        print()

In [26]:
calculate()

In [31]:
calculate.__name__

'calculate'